In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [3]:
import os
import random
from glob import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from model import Transformer
from utils import VectorizeChar, DisplayOutputs, CustomSchedule, path_to_features , wer, cer
import pandas as pd

In [4]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
def set_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_seeds()

In [6]:
vectorizer = VectorizeChar(100)

In [7]:
max_target_len = 100
print("vocab size", len(vectorizer.get_vocabulary()))

vocab size 95


In [8]:
def encode(path,txt):
  txt = txt.numpy().decode('utf8')
  y = tf.convert_to_tensor(vectorizer(txt),dtype=tf.int64)
  x = path_to_features(path)
  return x,y
def tf_encode(path,txt):
  x,y = tf.py_function(encode, [path,txt], [tf.float32,tf.int64])
  return x,y
def create_tf_dataset(data, batch_size=4):
  dataset = tf.data.Dataset.from_tensor_slices((np.array(data["filename"].values),np.array(data["transcript"].values)))
  dataset = dataset.map(tf_encode, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.padded_batch(batch_size, padded_shapes=([None,20], [None]))
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  dataset = dataset.map(lambda x, y: {"source": x, "target": y})
  return dataset

## Create & train the end-to-end model

In [9]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/iam_data.xlsx')
df["file_path"] = df["file_path"].apply(lambda x: x.replace("\\", "/"))
df['filename'] = df['file_path'].apply(lambda x: "/content/drive/MyDrive/Colab Notebooks/bin_files/"+x.split('/')[-1].replace('.xml','.bin'))
# split data into train and test
msk = np.random.rand(len(df)) < 0.95
train_data = df[msk]
dev_data = df[~msk]
# create tf dataset for training and validation
ds = create_tf_dataset(train_data, batch_size=32).cache().prefetch(tf.data.AUTOTUNE)
val_ds = create_tf_dataset(dev_data, batch_size=16).cache().prefetch(tf.data.AUTOTUNE)

In [10]:
len(train_data),len(dev_data)

(11609, 578)

In [11]:
# to display the output of the model after each epoch for the first batch of the validation set
batch = next(iter(val_ds))
# The vocabulary to convert predicted indices into characters
idx_to_char = vectorizer.get_vocabulary()
display_cb = DisplayOutputs(
    batch, idx_to_char, target_start_token_idx=2, target_end_token_idx=3
)

model = Transformer(
    num_hid=100,
    num_head=2,
    num_feed_forward=256,
    target_maxlen=max_target_len,
    num_layers_enc=4,
    num_layers_dec=1,
    num_classes=95,
)

loss_fn = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, label_smoothing=0.1,
)

learning_rate = CustomSchedule(
    init_lr=0.00001,
    lr_after_warmup=0.001,
    final_lr=0.00001,
    warmup_epochs=15,
    decay_epochs=85,
    steps_per_epoch=len(ds),
)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss=loss_fn)

In [12]:
#  train the model
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=20)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'transformer', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:667: UserWarning: `model.compiled_loss()` is deprecated. Instead, use `model.compute_loss(x, y, y_pred, sample_weight, training)`.
  warnings.warn(


363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - loss: 3.1034
=== Predictions after epoch 1 ===
target:     <and all were present at Rodney>
prediction: <thee s tae  a  the i   a a ion the the  ion in ior the  te  t iooe t t  t  e ta  o t t  t ie t t  t

target:     <slowed up the attack. I am>
prediction: <thee s tae  a  the i   a a ion the the  ion in ior the  te  t iooe t t  t  e ta  o t t  te t t t  t

target:     <Consequently the navy>
prediction: <thee s tae  a  the i   a a ion the the  ion in ior the  te  t iooe t t  t  e ta  o t t  t ie t t  t

target:     <John Conroy Out for the season. By>
prediction: <thee s tae  a  the i   a a ion the the  ion in ior the  te  t iooe t t  t  e ta  o t t  t ie t t  t

target:     <unguarded, Wills was able>
prediction: <the the ae  a  the i   a a ion the the  ion in ior the  te  t iooe t t  t  e ta  o t t  t ie t t  t

363/363 ━━━━━━━━━━━━━━━━━━━━ 3462s 9s/step - loss: 3.1028 - val_loss: 2.6094
Epoch 2/20
362/363 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step 

In [13]:
#  train the model
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=20)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.8334
=== Predictions after epoch 1 ===
target:     <and all were present at Rodney>
prediction: <andatll were resent at Mry>

target:     <slowed up the attack. I am>
prediction: <sloaded. The the attack. I >

target:     <Consequently the navy>
prediction: <Consedynenthey the havery>

target:     <John Conroy Out for the season. By>
prediction: <I ohn Corront for to the season By>

target:     <unguarded, Wills was able>
prediction: <mgararded, Will s was able>

363/363 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - loss: 0.8334 - val_loss: 0.9478
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.8123
=== Predictions after epoch 2 ===
target:     <and all were present at Rodney>
prediction: <and at were perest at of Mreeny>

target:     <slowed up the attack. I am>
prediction: <Doned. uppr the attach. I man>

target:     <Consequently the navy>
prediction: <Conseqenenthey the havery>

target:     <John Conroy Out for th

In [14]:
#  train the model
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=20)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.6543
=== Predictions after epoch 1 ===
target:     <and all were present at Rodney>
prediction: <ard attl were pert at Mrdy>

target:     <slowed up the attack. I am>
prediction: <slowed upr the attack. I am>

target:     <Consequently the navy>
prediction: <Conseqenently the way>

target:     <John Conroy Out for the season. By>
prediction: <John conroy cent for the season. By>

target:     <unguarded, Wills was able>
prediction: <magnarded, Willls was able>

363/363 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - loss: 0.6543 - val_loss: 0.8885
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.6486
=== Predictions after epoch 2 ===
target:     <and all were present at Rodney>
prediction: <aundedl were probert it Mr . >

target:     <slowed up the attack. I am>
prediction: <slowed upr the atttach I am>

target:     <Consequently the navy>
prediction: <Concegconnently the vary>

target:     <John Conroy Out for the season

In [15]:
#  train the model
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=20)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.5869
=== Predictions after epoch 1 ===
target:     <and all were present at Rodney>
prediction: <and att were pent at Mry >

target:     <slowed up the attack. I am>
prediction: <sload. upre attack. I am Iam>

target:     <Consequently the navy>
prediction: <Conceqeently the wasary>

target:     <John Conroy Out for the season. By>
prediction: <than Courrg Aor the season By>

target:     <unguarded, Wills was able>
prediction: <mgnarded, Wills was able>

363/363 ━━━━━━━━━━━━━━━━━━━━ 15s 40ms/step - loss: 0.5869 - val_loss: 0.9249
Epoch 2/20
362/363 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.5868
=== Predictions after epoch 2 ===
target:     <and all were present at Rodney>
prediction: <ard att were pent at wdrucy>

target:     <slowed up the attack. I am>
prediction: <slowed up the attack. I am>

target:     <Consequently the navy>
prediction: <Conceqeentsenty the vary>

target:     <John Conroy Out for the season. By>
pre

## Evalute the model

In [16]:
test_dataset = create_tf_dataset(dev_data, batch_size=32)
idx_to_char = vectorizer.get_vocabulary()


result=[]
for batch_idx, batch in enumerate( test_dataset):
  print(batch_idx)
  source = batch["source"]
  target = batch["target"].numpy()
  bs = tf.shape(source)[0]
  preds = model.generate(source, 2)
  preds = preds.numpy()
  for i in range(bs):
      target_text = "".join([idx_to_char[_] for _ in target[i, :]])
      prediction = ""
      for idx in preds[i, :]:
          prediction += idx_to_char[idx]
          if idx == 3:
              break
      result.append({"target":target_text.replace('-',''),"prediction":prediction})
      print(f"target:     {target_text.replace('-','')}")
      print(f"prediction: {prediction}\n")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [17]:
prediction_result = pd.DataFrame(result)
prediction_result.head()

,target,prediction
0,<and all were present at Rodney>,<and att weve perest ot Pody>
1,<slowed up the attack. I am>,<slowed uppr the athach. Iam>
2,<Consequently the navy>,"<Conceqeem, Hoj he way>"
3,<John Conroy Out for the season. By>,<J ohn Corroy Ont for the season. By>
4,"<unguarded, Wills was able>","<mgauarde , Wills was able>"


In [18]:
prediction_result['prediction'] = prediction_result['prediction'].apply(lambda x:x.replace('<','').replace('>',''))
prediction_result['target'] = prediction_result['target'].apply(lambda x:x.replace('<','').replace('>',''))

In [19]:
prediction_result['wer'] = prediction_result.apply(lambda x: wer(x.target, x.prediction) , axis=1)
prediction_result['cer'] = prediction_result.apply(lambda x: cer(x.target, x.prediction) , axis=1)
print(f" cer {prediction_result['cer'].mean():.5f} wer {prediction_result['wer'].mean():.5f}")

 cer 0.20287 wer 0.52231


In [20]:
prediction_result.to_excel('/content/drive/MyDrive/Colab Notebooks/Results/iam_results_03.xlsx')

In [21]:
#  Save the model
model.save_weights('/content/drive/MyDrive/Colab Notebooks/Models/iam888.weights.h5')